<a href="https://colab.research.google.com/github/Thomas-George-T/Retro-Reel-Recommender/blob/retro-api/Retro_Reel_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retrol Reel Recommender

## Data Acquisition

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# display full data in the cell without truncating
pd.set_option('display.max_colwidth', None)

In [3]:
!wget -q -O 'retro.json' https://prosona.b-cdn.net/Movies/catalog-v2.txt
print("Data Downloaded!")

Data Downloaded!


In [4]:
retro_raw = pd.read_json('retro.json')
retro_raw.head()

,Movies
0,"{'ID': 'b70eec12b79148dfbc97ac3b69ca49bf', 'Title': 'Angel On My Shoulder', 'ReleaseYear': 1946, 'Genres': ['Adventure', 'Comedy', 'Fantasy'], 'Synopsis': 'The Devil arranges for a deceased gangster to return to Earth as a well-respected judge to make up for his previous life.', 'Directors': ['Archie Mayo'], 'Cast': ['Paul Muni', 'Anne Baxter', 'Claude Rains'], 'Trivia': ['Director Archie Mayo clashed repeatedly with star Paul Muni during production. Muni didn't make another film for eight years.', 'The film's original title was ""Me and Satan"" but producer Charles R. Rogers decided to change it to Angel on My Shoulder (1946) on the conclusion that no one would want to go to see a film about the Devil.'], 'Keywords': 'Devil Soul angel on my shoulder 1946 adventure comedy fantasy archie mayo paul muni anne baxter claude rains', 'TrailerLink': 'https://prosona.b-cdn.net/Movies/Angel%20On%20My%20Shoulder%20%281946%29/Angel%20On%20My%20Shoulder%20%281946%29%20Trailer.mp4', 'MovieLink': 'https://prosona.b-cdn.net/Movies/Angel%20On%20My%20Shoulder%20%281946%29/Angel%20On%20My%20Shoulder%20%281946%29.mp4', 'HorizontalPosterLink': 'https://prosona.b-cdn.net/Movies/Angel%20On%20My%20Shoulder%20%281946%29/horizontal.jpg', 'VerticalPosterLink': 'https://prosona.b-cdn.net/Movies/Angel%20On%20My%20Shoulder%20%281946%29/vertical.jpg', 'SnapshotLink': 'https://prosona.b-cdn.net/Movies/Angel%20On%20My%20Shoulder%20%281946%29/screenshot.jpg', 'SubtitlesLink': '', 'Offset': 0.0}"
1,"{'ID': '1950e4a11304438dbfd4792a908fc76f', 'Title': 'Flirting with Fate', 'ReleaseYear': 1916, 'Genres': ['Adventure', 'Comedy'], 'Synopsis': 'In the midst of an emotional depression, a man hires a murderer to kill him. But the despair soon passes, and the man must now escape the killer he's hired to end his life.', 'Directors': ['Christy Cabanne'], 'Cast': ['Douglas Fairbanks', 'Jewel Carmen', 'Howard Gaye'], 'Trivia': [], 'Keywords': 'hitman assasin killer escape flirting with fate 1916 adventure comedy christy cabanne douglas fairbanks jewel carmen howard gaye', 'TrailerLink': 'https://prosona.b-cdn.net/Movies/Flirting%20With%20Fate%20%281916%29/preview.mp4', 'MovieLink': 'https://prosona.b-cdn.net/Movies/Flirting%20With%20Fate%20%281916%29/movie.mp4', 'HorizontalPosterLink': 'https://prosona.b-cdn.net/Movies/Flirting%20With%20Fate%20%281916%29/horizontal.png', 'VerticalPosterLink': 'https://prosona.b-cdn.net/Movies/Flirting%20With%20Fate%20%281916%29/vertical.png', 'SnapshotLink': 'https://prosona.b-cdn.net/Movies/Flirting%20With%20Fate%20%281916%29/screenshot.jpg', 'SubtitlesLink': '', 'Offset': 0.0}"
2,"{'ID': '37d8cae8d87f4dbc9cf8967454537afb', 'Title': 'Detour', 'ReleaseYear': 1945, 'Genres': ['Crime', 'Drama', 'Noir'], 'Synopsis': 'Chance events trap hitch-hiker Al Roberts in a tightening net of film noir trouble.', 'Directors': ['Edgar G. Ulmer'], 'Cast': ['Tom Neal', 'Ann Savage', 'Claudia Drake'], 'Trivia': ['The budget PRC gave director Edgar G. Ulmer for this film was so small that the 1941 Lincoln Continental V-12 convertible driven by Charles Haskell was actually Ulmer's personal car.', 'Ann Savage worked on her biography for the last decade of her life. It was released in early 2010 called ""Savage Detours.""', 'Shot quickly in mostly two locations: the hotel room apartment, and the car in front of a rear projection screen on a soundstage at PRC.'], 'Keywords': 'detour 1945 crime drama noir edgar g. ulmer tom neal ann savage claudia drake', 'TrailerLink': 'https://prosona.b-cdn.net/Movies/Detour%20-%201945/Detour%20-%201945%20Trailer.mp4', 'MovieLink': 'https://prosona.b-cdn.net/Movies/Detour%20-%201945/Detour-1945.mp4', 'HorizontalPosterLink': 'https://prosona.b-cdn.net/Movies/Detour%20-%201945/horizontal-poster.jpg', 'VerticalPosterLink': 'https://prosona.b-cdn.net/Movies/Detour%20-%201945/vertical-poster.jpg', 'SnapshotLink': 'https://prosona.b-cdn.net/Movies/Detour%20-%201945/snapshot.jpg', 'SubtitlesLink': 'https://prosona.b-cdn.net

## Data Preprocessing

In [5]:
retro_raw_flatten = pd.json_normalize(retro_raw['Movies'])
retro_raw_flatten.head()

,ID,Title,ReleaseYear,Genres,Synopsis,Directors,Cast,Trivia,Keywords,TrailerLink,MovieLink,HorizontalPosterLink,VerticalPosterLink,SnapshotLink,SubtitlesLink,Offset
0,b70eec12b79148dfbc97ac3b69ca49bf,Angel On My Shoulder,1946,"[Adventure, Comedy, Fantasy]",The Devil arranges for a deceased gangster to return to Earth as a well-respected judge to make up for his previous life.,[Archie Mayo],"[Paul Muni, Anne Baxter, Claude Rains]","[Director Archie Mayo clashed repeatedly with star Paul Muni during production. Muni didn't make another film for eight years., The film's original title was ""Me and Satan"" but producer Charles R. Rogers decided to change it to Angel on My Shoulder (1946) on the conclusion that no one would want to go to see a film about the Devil.]",Devil Soul angel on my shoulder 1946 adventure comedy fantasy archie mayo paul muni anne baxter claude rains,https://prosona.b-cdn.net/Movies/Angel%20On%20My%20Shoulder%20%281946%29/Angel%20On%20My%20Shoulder%20%281946%29%20Trailer.mp4,https://prosona.b-cdn.net/Movies/Angel%20On%20My%20Shoulder%20%281946%29/Angel%20On%20My%20Shoulder%20%281946%29.mp4,https://prosona.b-cdn.net/Movies/Angel%20On%20My%20Shoulder%20%281946%29/horizontal.jpg,https://prosona.b-cdn.net/Movies/Angel%20On%20My%20Shoulder%20%281946%29/vertical.jpg,https://prosona.b-cdn.net/Movies/Angel%20On%20My%20Shoulder%20%281946%29/screenshot.jpg,,0.0
1,1950e4a11304438dbfd4792a908fc76f,Flirting with Fate,1916,"[Adventure, Comedy]","In the midst of an emotional depression, a man hires a murderer to kill him. But the despair soon passes, and the man must now escape the killer he's hired to end his life.",[Christy Cabanne],"[Douglas Fairbanks, Jewel Carmen, Howard Gaye]",[],hitman assasin killer escape flirting with fate 1916 adventure comedy christy cabanne douglas fairbanks jewel carmen howard gaye,https://prosona.b-cdn.net/Movies/Flirting%20With%20Fate%20%281916%29/preview.mp4,https://prosona.b-cdn.net/Movies/Flirting%20With%20Fate%20%281916%29/movie.mp4,https://prosona.b-cdn.net/Movies/Flirting%20With%20Fate%20%281916%29/horizontal.png,https://prosona.b-cdn.net/Movies/Flirting%20With%20Fate%20%281916%29/vertical.png,https://prosona.b-cdn.net/Movies/Flirting%20With%20Fate%20%281916%29/screenshot.jpg,,0.0
2,37d8cae8d87f4dbc9cf8967454537afb,Detour,1945,"[Crime, Drama, Noir]",Chance events trap hitch-hiker Al Roberts in a tightening net of film noir trouble.,[Edgar G. Ulmer],"[Tom Neal, Ann Savage, Claudia Drake]","[The budget PRC gave director Edgar G. Ulmer for this film was so small that the 1941 Lincoln Continental V-12 convertible driven by Charles Haskell was actually Ulmer's personal car., Ann Savage worked on her biography for the last decade of her life. It was released in early 2010 called ""Savage Detours."", Shot quickly in mostly two locations: the hotel room apartment, and the car in front of a rear projection screen on a soundstage at PRC.]",detour 1945 crime drama noir edgar g. ulmer tom neal ann savage claudia drake,https://prosona.b-cdn.net/Movies/Detour%20-%201945/Detour%20-%201945%20Trailer.mp4,https://prosona.b-cdn.net/Movies/Detour%20-%201945/Detour-1945.mp4,https://prosona.b-cdn.net/Movies/Detour%20-%201945/horizontal-poster.jpg,https://prosona.b-cdn.net/Movies/Detour%20-%201945/vertical-poster.jpg,https://prosona.b-cdn.net/Movies/Detour%20-%201945/snapshot.jpg,https://prosona.b-cdn.net/Movies/Detour%20-%201945/sub%20EN.srt,0.0
3,b99dd03c00cc4fb9b2d4ad26082de498,The Big Trees,1952,"[Action, Romance, Western]",A Quaker colony tries to save the giant sequoias from a timber baron.,[Felix E. Feist],"[Kirk Douglas, Eve Miller, Patrice Wymore]","[According to Kirk Douglas in his autobiography ""The Ragman's Son"", he agreed to act in this film for free, in order to end his contract with Warner Bros., The reading that Fallon does at the dinner is from Psalms 41, verse 1., In the same year, Kirk Douglas starred in a movie titled The Big Trees and a movie titled The Big Sky. Though they were narra

In [6]:
retro_raw_flatten.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID                    49 non-null     object 
 1   Title                 49 non-null     object 
 2   ReleaseYear           49 non-null     int64  
 3   Genres                49 non-null     object 
 4   Synopsis              49 non-null     object 
 5   Directors             49 non-null     object 
 6   Cast                  49 non-null     object 
 7   Trivia                49 non-null     object 
 8   Keywords              49 non-null     object 
 9   TrailerLink           49 non-null     object 
 10  MovieLink             49 non-null     object 
 11  HorizontalPosterLink  49 non-null     object 
 12  VerticalPosterLink    49 non-null     object 
 13  SnapshotLink          49 non-null     object 
 14  SubtitlesLink         49 non-null     object 
 15  Offset                49 

## Feature Selection

We chose ID, Title, ReleaseYear, Genres, Synopsis, Directors and Cast as our features. They are to generate a bag of keywords which we will use as our basis of the cosimilarity matrix. 

In [7]:
df4 = retro_raw_flatten [['ID','Title','ReleaseYear','Genres','Synopsis','Directors','Cast']]
df4

,ID,Title,ReleaseYear,Genres,Synopsis,Directors,Cast
0,b70eec12b79148dfbc97ac3b69ca49bf,Angel On My Shoulder,1946,"[Adventure, Comedy, Fantasy]",The Devil arranges for a deceased gangster to return to Earth as a well-respected judge to make up for his previous life.,[Archie Mayo],"[Paul Muni, Anne Baxter, Claude Rains]"
1,1950e4a11304438dbfd4792a908fc76f,Flirting with Fate,1916,"[Adventure, Comedy]","In the midst of an emotional depression, a man hires a murderer to kill him. But the despair soon passes, and the man must now escape the killer he's hired to end his life.",[Christy Cabanne],"[Douglas Fairbanks, Jewel Carmen, Howard Gaye]"
2,37d8cae8d87f4dbc9cf8967454537afb,Detour,1945,"[Crime, Drama, Noir]",Chance events trap hitch-hiker Al Roberts in a tightening net of film noir trouble.,[Edgar G. Ulmer],"[Tom Neal, Ann Savage, Claudia Drake]"
3,b99dd03c00cc4fb9b2d4ad26082de498,The Big Trees,1952,"[Action, Romance, Western]",A Quaker colony tries to save the giant sequoias from a timber baron.,[Felix E. Feist],"[Kirk Douglas, Eve Miller, Patrice Wymore]"
4,5566ce75b5954f3dad323c2f154856a3,The Jungle Book,1942,"[Action, Adventure]",\r\nA boy raised by wild animals tries to adapt to human village life.,[Zoltan Korda],"[Sabu, Joseph Calleia, John Qualen]"
5,62e08996bb034303a417d10e136bedda,The Lost World,1925,"[Action, Fantasy, Thriller]",The first film adaptation of Sir Arthur Conan Doyle's classic novel about a land where prehistoric creatures still roam.,[Harry O. Hoyt],"[Wallace Beery, Bessie Love, Lloyd Hughes]"
6,74ac96777a4d404b8eba028e44f8f06b,Suddenly,1954,"[Crime, Drama, Noir]","In the city of Suddenly, three gangsters trap the Benson family in their own house, on the top of a hill nearby the railroad station, with the intention of killing the president of the USA.",[],[]
7,4014bc52406d47d88f077ab9ca6e988c,Bloody Pit Of Horror,1965,"[Horror, Thriller]","A photographer and his models go to an old, abandoned castle to shoot some sexy covers for horror novels. Unbeknownst to them, the castle is inhabited by a lunatic who believes himself to be the reincarnated spirit of a 17th-century executioner whose job it is to protect the castle against intruders.",[Domenico Massimo],"[Mickey Hargitay, Walter Brandi, Luisa Baratto]"
8,97063c49fbe94966a7f883594896855b,Charlie Chaplin's The Rink,1916,"[Short, Comedy]","After causing restaurant chaos at work, a bumbling waiter tears up the local roller rink with his skating.",[Charles Chaplin],"[Charles Chaplin, Edna Purviance, James T. Kelley]"
9,3cb4bd7eed36485c8faf54c41c243f20,Texas Terror,1935,"[Action, Adventure, Drama]","Sheriff John Higgins quits and goes into prospecting after he thinks he has killed his best friend in shooting it out with robbers. He encounters his dead buddy's daughter, who has come from back east, and helps her run her ranch. Then she finds out about his past.",[Robert N. Bradbury],"[John Wayne, Lucile Browne, LeRoy Mason]"


In [8]:
df4['Synopsis']

0                                                                                                                                                                                                                                                 The Devil arranges for a deceased gangster to return to Earth as a well-respected judge to make up for his previous life.
1                                                                                                                                                                                              In the midst of an emotional depression, a man hires a murderer to kill him. But the despair soon passes, and the man must now escape the killer he's hired to end his life.
2                                                                                                                                                                                                                                                                               

## Data Processing

Using regular expression library, remove special characters and punctuations from the columns and make them into lower case while at it. This is so as to generate the keywords more easily. We remove the space between the names to avoid the scenario in the cosimilarity index where the cast/director have the same first name but different last name and the similarity index gets skewed.


In [9]:
import re

#set up punctuations to be replaced
remove_punct_space = re.compile("(\.)|(\')|(\s)")
remove_punct = re.compile("(\.)|(\')|(\:)|(\,)|(\-)|(\$)|(\")")

In [10]:
df4.iloc[4,4] = df4.iloc[4,4].replace('\r\n','')
df4['Directors'] = df4['Directors'].map(lambda x : [ remove_punct_space.sub("", y.lower()) for y in x ])
df4['Cast'] = df4['Cast'].map(lambda x : [ remove_punct_space.sub("", y.lower()) for y in x ])

df4['Synopsis'] = df4['Synopsis'].map(lambda x: remove_punct.sub("", x.lower()))
df4['Title'] = df4['Title'].map(lambda x: remove_punct.sub("", x.lower()))

# Make the rest of the columns also lower case
df4['Genres'] = df4['Genres'].map(lambda x: [ y.lower() for y in x ])
df4.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.

,ID,Title,ReleaseYear,Genres,Synopsis,Directors,Cast
0,b70eec12b79148dfbc97ac3b69ca49bf,angel on my shoulder,1946,"[adventure, comedy, fantasy]",the devil arranges for a deceased gangster to return to earth as a wellrespected judge to make up for his previous life,[archiemayo],"[paulmuni, annebaxter, clauderains]"
1,1950e4a11304438dbfd4792a908fc76f,flirting with fate,1916,"[adventure, comedy]",in the midst of an emotional depression a man hires a murderer to kill him but the despair soon passes and the man must now escape the killer hes hired to end his life,[christycabanne],"[douglasfairbanks, jewelcarmen, howardgaye]"
2,37d8cae8d87f4dbc9cf8967454537afb,detour,1945,"[crime, drama, noir]",chance events trap hitchhiker al roberts in a tightening net of film noir trouble,[edgargulmer],"[tomneal, annsavage, claudiadrake]"
3,b99dd03c00cc4fb9b2d4ad26082de498,the big trees,1952,"[action, romance, western]",a quaker colony tries to save the giant sequoias from a timber baron,[felixefeist],"[kirkdouglas, evemiller, patricewymore]"
4,5566ce75b5954f3dad323c2f154856a3,the jungle book,1942,"[action, adventure]",a boy raised by wild animals tries to adapt to human village life,[zoltankorda],"[sabu, josephcalleia, johnqualen]"


## NLP
We will generate keywords from the synopsis column using rake from nltk. Ultimately, we will use all the columns to generate the bag of words.

In [11]:
!pip install nltk
from nltk.corpus import stopwords 

In [12]:
!pip install rake-nltk
from rake_nltk import Rake

  Created wheel for rake-nltk: filename=rake_nltk-1.0.4-py2.py3-none-any.whl size=7819 sha256=a531c53a23194dda65b4f2837c6c1701ee4c55b3e6909c9ac5c78e7ee6d5a1de
  Stored in directory: /root/.cache/pip/wheels/ef/92/fc/271b3709e71a96ffe934b27818946b795ac6b9b8ff8682483f
Successfully built rake-nltk


In [13]:
# initializing the new column
df4['Key_words'] = ""

for index, row in df4.iterrows():
    synopsis = row['Synopsis']
    
    # instantiating Rake, by default it uses english stopwords from NLTK and discards all puntuation
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(synopsis)

    # getting the dictionary whith key words and their scores
    key_words_dict_scores = r.get_word_degrees()

    # assigning the key words to the new column
    row['Key_words'] = list(key_words_dict_scores.keys())
    df4['Key_words'][index] = row['Key_words']
       

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, se

In [14]:
df5 = df4['ID']
df5.head()

0    b70eec12b79148dfbc97ac3b69ca49bf
1    1950e4a11304438dbfd4792a908fc76f
2    37d8cae8d87f4dbc9cf8967454537afb
3    b99dd03c00cc4fb9b2d4ad26082de498
4    5566ce75b5954f3dad323c2f154856a3
Name: ID, dtype: object

In [15]:
df4.drop(columns=['ID','Synopsis'], inplace=True)
df4.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Title,ReleaseYear,Genres,Directors,Cast,Key_words
0,angel on my shoulder,1946,"[adventure, comedy, fantasy]",[archiemayo],"[paulmuni, annebaxter, clauderains]","[devil, arranges, return, wellrespected, judge, deceased, gangster, previous, life, make, earth]"
1,flirting with fate,1916,"[adventure, comedy]",[christycabanne],"[douglasfairbanks, jewelcarmen, howardgaye]","[escape, man, hires, must, killer, hes, hired, end, life, emotional, depression, despair, soon, passes, murderer, midst, kill]"
2,detour,1945,"[crime, drama, noir]",[edgargulmer],"[tomneal, annsavage, claudiadrake]","[film, noir, trouble, chance, events, trap, hitchhiker, al, roberts, tightening, net]"
3,the big trees,1952,"[action, romance, western]",[felixefeist],"[kirkdouglas, evemiller, patricewymore]","[save, giant, sequoias, quaker, colony, tries, timber, baron]"
4,the jungle book,1942,"[action, adventure]",[zoltankorda],"[sabu, josephcalleia, johnqualen]","[boy, raised, wild, animals, tries, adapt, human, village, life]"


In [16]:
df4['ReleaseYear'] = df4['ReleaseYear'].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
df4['bag_of_words'] = ''
columns = df4.columns
for index, row in df4.iterrows():
    words = ''
    for col in columns:
        if col != 'ReleaseYear' and col != 'Title':
            words = words + ' '.join(row[col])+ ' '
        else:
            words = words + row[col]+ ' '
    row['bag_of_words'] = words
    
df4['bag_of_words'].head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0                              angel on my shoulder 1946 adventure comedy fantasy archiemayo paulmuni annebaxter clauderains devil arranges return wellrespected judge deceased gangster previous life make earth  
1    flirting with fate 1916 adventure comedy christycabanne douglasfairbanks jewelcarmen howardgaye escape man hires must killer hes hired end life emotional depression despair soon passes murderer midst kill  
2                                                               detour 1945 crime drama noir edgargulmer tomneal annsavage claudiadrake film noir trouble chance events trap hitchhiker al roberts tightening net  
3                                                                  the big trees 1952 action romance western felixefeist kirkdouglas evemiller patricewymore save giant sequoias quaker colony tries timber baron  
4                                                                          the jungle book 1942 action adventure zoltankorda sabu josephcalleia johnqual

In [18]:
df4.head()

,Title,ReleaseYear,Genres,Directors,Cast,Key_words,bag_of_words
0,angel on my shoulder,1946,"[adventure, comedy, fantasy]",[archiemayo],"[paulmuni, annebaxter, clauderains]","[devil, arranges, return, wellrespected, judge, deceased, gangster, previous, life, make, earth]",angel on my shoulder 1946 adventure comedy fantasy archiemayo paulmuni annebaxter clauderains devil arranges return wellrespected judge deceased gangster previous life make earth
1,flirting with fate,1916,"[adventure, comedy]",[christycabanne],"[douglasfairbanks, jewelcarmen, howardgaye]","[escape, man, hires, must, killer, hes, hired, end, life, emotional, depression, despair, soon, passes, murderer, midst, kill]",flirting with fate 1916 adventure comedy christycabanne douglasfairbanks jewelcarmen howardgaye escape man hires must killer hes hired end life emotional depression despair soon passes murderer midst kill
2,detour,1945,"[crime, drama, noir]",[edgargulmer],"[tomneal, annsavage, claudiadrake]","[film, noir, trouble, chance, events, trap, hitchhiker, al, roberts, tightening, net]",detour 1945 crime drama noir edgargulmer tomneal annsavage claudiadrake film noir trouble chance events trap hitchhiker al roberts tightening net
3,the big trees,1952,"[action, romance, western]",[felixefeist],"[kirkdouglas, evemiller, patricewymore]","[save, giant, sequoias, quaker, colony, tries, timber, baron]",the big trees 1952 action romance western felixefeist kirkdouglas evemiller patricewymore save giant sequoias quaker colony tries timber baron
4,the jungle book,1942,"[action, adventure]",[zoltankorda],"[sabu, josephcalleia, johnqualen]","[boy, raised, wild, animals, tries, adapt, human, village, life]",the jungle book 1942 action adventure zoltankorda sabu josephcalleia johnqualen boy raised wild animals tries adapt human village life


In [19]:
df4['ID'] = df5
df4=df4[['ID','Title','ReleaseYear','Genres','Directors','Cast','bag_of_words']]
df4.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,ID,Title,ReleaseYear,Genres,Directors,Cast,bag_of_words
0,b70eec12b79148dfbc97ac3b69ca49bf,angel on my shoulder,1946,"[adventure, comedy, fantasy]",[archiemayo],"[paulmuni, annebaxter, clauderains]",angel on my shoulder 1946 adventure comedy fantasy archiemayo paulmuni annebaxter clauderains devil arranges return wellrespected judge deceased gangster previous life make earth
1,1950e4a11304438dbfd4792a908fc76f,flirting with fate,1916,"[adventure, comedy]",[christycabanne],"[douglasfairbanks, jewelcarmen, howardgaye]",flirting with fate 1916 adventure comedy christycabanne douglasfairbanks jewelcarmen howardgaye escape man hires must killer hes hired end life emotional depression despair soon passes murderer midst kill
2,37d8cae8d87f4dbc9cf8967454537afb,detour,1945,"[crime, drama, noir]",[edgargulmer],"[tomneal, annsavage, claudiadrake]",detour 1945 crime drama noir edgargulmer tomneal annsavage claudiadrake film noir trouble chance events trap hitchhiker al roberts tightening net
3,b99dd03c00cc4fb9b2d4ad26082de498,the big trees,1952,"[action, romance, western]",[felixefeist],"[kirkdouglas, evemiller, patricewymore]",the big trees 1952 action romance western felixefeist kirkdouglas evemiller patricewymore save giant sequoias quaker colony tries timber baron
4,5566ce75b5954f3dad323c2f154856a3,the jungle book,1942,"[action, adventure]",[zoltankorda],"[sabu, josephcalleia, johnqualen]",the jungle book 1942 action adventure zoltankorda sabu josephcalleia johnqualen boy raised wild animals tries adapt human village life




---



In [20]:
import pickle

In [21]:
pickle.dump(df4, open('movies.pickle', 'wb'))

## Using TF_IDF

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df4['bag_of_words'])

In [23]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.04898959, 0.        , ..., 0.03715558, 0.01311707,
        0.01276385],
       [0.04898959, 1.        , 0.        , ..., 0.03458854, 0.01221082,
        0.04187143],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.00989937],
       ...,
       [0.03715558, 0.03458854, 0.        , ..., 1.        , 0.01362476,
        0.01325787],
       [0.01311707, 0.01221082, 0.        , ..., 0.01362476, 1.        ,
        0.01266281],
       [0.01276385, 0.04187143, 0.00989937, ..., 0.01325787, 0.01266281,
        1.        ]])

In [24]:
pickle.dump(tfidf_matrix, open('engine.pickle', 'wb'))



---



## Testing the `recommender.py` used in the API call.

In [25]:
df = pickle.load(open('movies.pickle', 'rb'))

In [26]:
tfidf_matrix = pickle.load(open('engine.pickle', 'rb'))

In [27]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

def recommend_mov(movieId, df, tfidf_matrix):
    
    try:
        indices = pd.Series(df.ID)
        # getting the index of the movie that matches the title
        idx = indices[indices == movieId].index[0]

    except:
        return 'MovieID not found, Please try again'
   
    recommended_movies = []
    
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    
    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df.ID)[i])     
      
    return recommended_movies

In [28]:
recommend_mov("b70eec12b79148dfbc97ac3b69ca49bf", df, tfidf_matrix)

['1950e4a11304438dbfd4792a908fc76f',
 '5566ce75b5954f3dad323c2f154856a3',
 'fe5901c51ca44c8cb40507949b2f2da0',
 '53461fabeaef483e942c9aaad235ca4d',
 '62e08996bb034303a417d10e136bedda',
 '41a069f0d71042be908429adf7e1324e',
 '0254461677cf4d0aa48319e226a68ca5',
 '62b36e28f1d6443c8bef36f7cd216a7d',
 '19eff70e96924a6496635736387e6fc5',
 'd3ae2ba361ca496b82c5622c8b626277']